In [1]:
!pip install pyspark==3.4.0 findspark pyarrow fastparquet

# Import findspark and initialise.
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 74.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317122 sha256=6892cb3304868113557ef801e98a70217f77a6b4f60b3c1a0c4864aef8a2c464
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# # 2. Create a temporary view of the DataFrame.
# Use wget to download the CSV file
!wget {url} -O home_sales_revised.csv

# Read the downloaded CSV file into a DataFrame
df = spark.read.csv("home_sales_revised.csv", header=True, inferSchema=True)

# Creating a temporary view
df.createOrReplaceTempView("home_sales")

# Show top 5 rows of dataframe
df.show(5)



--2023-11-26 11:16:43--  https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv
Resolving 2u-data-curriculum-team.s3.amazonaws.com (2u-data-curriculum-team.s3.amazonaws.com)... 54.231.235.73, 3.5.28.218, 3.5.16.211, ...
Connecting to 2u-data-curriculum-team.s3.amazonaws.com (2u-data-curriculum-team.s3.amazonaws.com)|54.231.235.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2747874 (2.6M) [text/csv]
Saving to: ‘home_sales_revised.csv’

home_sales_revised. 100%[===================>]   2.62M  6.15MB/s    in 0.4s    

2023-11-26 11:16:44 (6.15 MB/s) - ‘home_sales_revised.csv’ saved [2747874/2747874]

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
query = """
    SELECT year, ROUND(AVG(price), 2) AS avg_price
    FROM (
        SELECT *, YEAR(date) AS year
        FROM home_sales
        WHERE bedrooms = 4
    ) AS four_bedroom_sales
    GROUP BY year
    ORDER BY year
"""

result = spark.sql(query)
result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query = """
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
"""

result = spark.sql(query)
result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
    SELECT date_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
"""

result = spark.sql(query)
result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
"""

result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 399856.95|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 401419.75|
|  81|1053472.79|
|  28| 402124.62|
|  76|1058802.78|
|  26| 401506.97|
|  27| 399537.66|
|  44| 400598.05|
|  12| 401501.32|
|  91|1137372.73|
|  22| 402022.68|
|  93|1026006.06|
|  47|  398447.5|
|   1| 401044.25|
|  52| 733780.26|
|  13| 398917.98|
+----+----------+
only showing top 20 rows

--- 0.9360013008117676 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
df.cache()


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
"""

result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 2.5433149337768555 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Specify the output directory for the Parquet files
output_directory = "parquet"

# Write the DataFrame as a partitioned Parquet file with overwrite mode
df.write.partitionBy("date_built").mode("overwrite").parquet(output_directory)

In [13]:
# 11. Read the formatted parquet data.
parquet_df = spark.read.parquet(output_directory)

# Show the first few rows of the DataFrame
parquet_df.show(5)

+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [14]:
# 12. Create a temporary table for the parquet data.
# Specify the temporary table name
parquet_temp_table = "parquet_table"

# Create a temporary table for the Parquet data
parquet_df.createOrReplaceTempView(parquet_temp_table)

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM parquet_table
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view
"""

result = spark.sql(query)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 0.9879515171051025 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
# Specify the temporary table name
home_sales_temp_table = "home_sales"

# Uncache the temporary table
spark.catalog.uncacheTable(home_sales_temp_table)

In [17]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached(home_sales_temp_table)

# Print the result
print(f"The table {home_sales_temp_table} is cached: {is_cached}")


The table home_sales is cached: False
